In [2]:
import time
import adodbapi
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [3]:
start_time = time.time()

In [4]:
ManufOrTopC ="Top Companies" #Manufacturer or Top Companies
BrandOrTopB= "Top Brands" #Brand or Top Brands
prodORitem = "Business Name"

client_manuf =["Malard Nicolas"]
client_brands = ["Nicolas"]

decimals = 2
sign = "after"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

categories =["Effervescents"]
sectors = ["Champagnes","Mousseux","Effervescents Sans Alcool"]
segments = ["Blanc De Blancs","Bruts","Crus","Extra Brut","Millesimes","Rosés","Mousseux Bourgogne","Mousseux Italiens","Mousseux Val De Loire","Mousseux Alsace","Mousseux Bordelais"]
subsegments= []
subcategories= []

national = False
customareas= "Revenue"
areas = ["CHANNEL","REGIONS", f'{customareas}']
 
regions_RET  = ["NICOLAS"]
channels_RET = []
market_RET = []
 
regions_CHAN = []
channels_CHAN = ["NICOLAS QCN","NICOLAS VCN","NICOLAS QCT","NICOLAS QCA","NICOLAS CCP"]
market_CHAN = []

regions_REG = []
channels_REG = ["NICOLAS IDF","NICOLAS PAC","NICOLAS RHO","NICOLAS AQU","NICOLAS EST"]
market_REG = []

regions_CUST = []
channels_CUST = ["CA A","CA B","CA C","CA D","CA E"]
market_CUST = []

data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending June 2025"
years = ['2023', '2024','2025']
end_date = "2025-07-01"
unit = "‘000000s"
unitDeviation = 1000000
valueToReplace = {}
past_12_months = pd.date_range(end=end_date, periods=12, freq='ME').strftime('%b-%y').tolist()
regions = regions_RET + regions_CHAN + regions_CUST + regions_REG
channels = channels_RET + channels_CHAN + channels_CUST + channels_REG
markets = market_RET + market_CHAN + market_CUST + market_REG

entity_hierarchy = [
    ("Region", regions),
    ("Channel", channels),
    ("Market", markets)
]
hierarchy_levels = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories)
]
server = "powerbi://api.powerbi.com/v1.0/myorg/Nicolas"
dataset_name = "Nicolas Sparkling Dataset"
conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};Timeout=600;"


path=os.path.join(os.getcwd(),"Assortment Datasets NewEX")

# Format months for DAX
p12m_dax = "{" + ", ".join(f'"{date}"' for date in past_12_months) + "}"

In [5]:
entity_hierarchy

[('Region', ['NICOLAS']),
 ('Channel',
  ['NICOLAS QCN',
   'NICOLAS VCN',
   'NICOLAS QCT',
   'NICOLAS QCA',
   'NICOLAS CCP',
   'CA A',
   'CA B',
   'CA C',
   'CA D',
   'CA E',
   'NICOLAS IDF',
   'NICOLAS PAC',
   'NICOLAS RHO',
   'NICOLAS AQU',
   'NICOLAS EST']),
 ('Market', [])]

In [6]:
# Function to execute DAX queries
def execute_dax_query(BrandorManuf,entity_name, area, hierby, entity_type):
    outputdic = {}
    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{BrandorManuf}],   
            "Value Share", IF(ISBLANK([Value Share]),0,[Value Share]),
            "SKU Share", IF(ISBLANK([SKU Share]),0,[SKU Share])
        ),
        Products[{hierby}] = "{entity_type}",
        TREATAS(
            {p12m_dax},
            Calendar[MonthYear]
        ),
        TREATAS(
            {{"{entity_name}"}},
            Market[{area}]
        ),
        FILTER(
            'Scope', 
            'Scope'[Scope] = "{hierby}"   
        )
    )   
    """
    try:
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                columns = [desc[0] for desc in cursor.description]
                data = cursor.fetchall()
                
                df = pd.DataFrame(data, columns=columns)
                df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
                df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
                grand_total_values = []
                for col in df.select_dtypes(include='number').columns:
                    if "Price" in col:
                        grand_total_values.append(df[col].mean())  # Take average for price columns
                    else:
                        grand_total_values.append(df[col].sum())   # Take sum for other columns
                grand_total_row = pd.DataFrame([["Grand Total"] + grand_total_values], columns=df.columns)
                df = pd.concat([df, grand_total_row], ignore_index=True)
                
                outputdic[f"{entity_type} | {entity_name}"] = df
                print(f"Query executed successfully for {entity_type} {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")
    return outputdic

assortmentbrand = {}
# **Threaded Execution**
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    for hierby, hier_values in hierarchy_levels:
        if isinstance(hier_values, list):
            for value in hier_values:
                for area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                            futures.append(executor.submit(execute_dax_query, "Top Brands",entity, area, hierby, value))

    # Wait for all tasks to complete
    for future in futures:
        result=future.result()
        assortmentbrand.update(result)
pd.to_pickle(assortmentbrand, os.path.join(path,"assortmentbybrand.pkl"))

assortmentmanuf = {}
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            futures.append(executor.submit(execute_dax_query, f"{ManufOrTopC}",entity, area, hierby, value))
    
    # Wait for all tasks to complete
    for future in futures:
        result=future.result()
        assortmentmanuf.update(result)

pd.to_pickle(assortmentmanuf,os.path.join(path,"assortmentbymanuf.pkl"))
# print(f"All DataFrames saved to { r"C:\Users\aleaa\Desktop\New Extraction\assortmentmanuf.pkl"}.")


Query executed successfully for Effervescents NICOLAS CCP.
Query executed successfully for Effervescents NICOLAS QCT.
Query executed successfully for Effervescents NICOLAS QCA.
Query executed successfully for Effervescents CA A.
Query executed successfully for Effervescents NICOLAS VCN.
Query executed successfully for Effervescents NICOLAS.
Query executed successfully for Effervescents NICOLAS QCN.
Query executed successfully for Effervescents CA B.
Query executed successfully for Effervescents CA C.
Query executed successfully for Effervescents NICOLAS IDF.
Query executed successfully for Effervescents CA D.
Query executed successfully for Effervescents NICOLAS PAC.
Query executed successfully for Effervescents CA E.
Query executed successfully for Effervescents NICOLAS RHO.
Query executed successfully for Effervescents NICOLAS AQU.
Query executed successfully for Champagnes NICOLAS.
Query executed successfully for Effervescents NICOLAS EST.
Query executed successfully for Champagnes 

In [ ]:
def execute_dax_query_with_retry(BrandorManuf, entity_name, area, hierby, entity_type, retries=3, delay=5):
    """
    Executes a DAX query with retry logic for handling transient errors.
    """
    for attempt in range(retries):
        try:
            outputdic = {}
            dax_query = f"""
            EVALUATE
            CALCULATETABLE(
                SUMMARIZE(
                    Products,
                    Products[{BrandorManuf}],  
                    Products[{prodORitem}],   
                    "Value Share", IF(ISBLANK([Value Share]), 0, [Value Share]),
                    "WD", IF(ISBLANK([WD]), 0, [WD]),
                    "Net Sales", IF(ISBLANK([Net Sales]), 0, [Net Sales]),
                    "Trade Margin %", IF(ISBLANK([Trade Margin %]), 0, [Trade Margin %]),
                    "{prodORitem} Sales Rate", IF(ISBLANK([{prodORitem} Sales Rate]), 0, [{prodORitem} Sales Rate])

                ),
                Products[Category] = "{categories[0]}",
                Products[{hierby}] = "{entity_type}",
                TREATAS(
                    {p12m_dax},
                    Calendar[MonthYear]
                ),
                TREATAS(
                    {{"{entity_name}"}},
                    Market[{area}]
                ),
                FILTER(
                    'Scope', 
                    'Scope'[Scope] = "{hierby}"   
                )
            )   
            """
            with adodbapi.connect(conn_str) as conn:
                conn.timeout = 300  # Increase timeout to 300 seconds
                with conn.cursor() as cursor:
                    cursor.execute(dax_query)
                    columns = [desc[0] for desc in cursor.description]
                    data = cursor.fetchall()
                    
                    df = pd.DataFrame(data, columns=columns)
                    df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
                    df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

                    # Add Grand Total row
                    if not df.empty:
                        numeric_cols = df.select_dtypes(include='number').columns

                        # Identify columns containing "price" (case insensitive)
                        price_cols = [col for col in numeric_cols if "price" in col.lower()]
                        sum_cols = [col for col in numeric_cols if col not in price_cols]

                        # Compute the grand total row
                        grand_total_values = {
                            col: df[col].mean() if col in price_cols else df[col].sum() 
                            for col in numeric_cols
                        }

                        # Create a DataFrame row for the grand total
                        grand_total_row = pd.DataFrame(
                            [["Grand Total"] + [grand_total_values.get(col, None) for col in df.columns[1:]]], 
                            columns=df.columns
                        )

                        # Append the Grand Total row to the DataFrame
                        df = pd.concat([df, grand_total_row], ignore_index=True)

                    outputdic[f"{entity_type} | {entity_name}"] = df
                    print(f"Query executed successfully for {entity_type} {entity_name}.")
                    return outputdic
        except adodbapi.DatabaseError as db_error:
            print(f"Attempt {attempt + 1} failed for {entity_type} {entity_name}: {db_error}")
            if attempt < retries - 1:
                time.sleep(delay * (attempt + 1))  # Exponential backoff
            else:
                print(f"Max retries reached for {entity_type} {entity_name}.")
                raise
        except Exception as e:
            print(f"Unexpected error for {entity_type} {entity_name}: {e}")
            raise

def process_entities(BrandorManuf, output_file, max_workers=4):
    """
    Processes entities in parallel and saves the results to a pickle file.
    """

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        future_to_key = {}
        dfs_results = {}
        ordered_keys = []
        result_dict = {}

        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                                    key = f"{value} | {entity}"
                                    ordered_keys.append(key)
                                    future = executor.submit(
                                        execute_dax_query_with_retry, BrandorManuf, entity, area, hierby, value
                                    )
                                    futures.append(future)
                                    future_to_key[future] = key

                
        
    temp_results = {}
    for future in as_completed(future_to_key):
        key = future_to_key[future]
        try:
            result = future.result()
            temp_results.update(result)
        except Exception as e:
            print(f"Failed to retrieve result for {key}: {e}")

    # Preserve original key order
    for key in ordered_keys:
        if key in temp_results:
            dfs_results[key] = temp_results[key]

    output_file = os.path.join(path, output_file)
    with open(output_file, "wb") as f:
        pd.to_pickle(dfs_results, f)

# Main execution
if __name__ == "__main__":
    # Process Brand or TopB entities
    process_entities(f"{BrandOrTopB}", "assortment.pkl", max_workers=8)

    # Process Manufacturer or TopC entities
    process_entities(f"{ManufOrTopC}", "assortment_manuf.pkl", max_workers=8)

Attempt 1 failed for Effervescents NICOLAS CCP: (-2147352567, 'Exception occurred.', (0, 'Microsoft OLE DB Provider for Analysis Services.', "Query (12, 92) Column '<oii>Business Name Sales Rate</oii>' cannot be found or may not be used in this expression. \n \nTechnical Details: \nRootActivityId: 62de1b9c-db4a-45cb-8b24-2caa3826fa9a \nDate (UTC): 8/26/2025 7:54:21 PM", None, 0, -2147467259), None)
Command:

            EVALUATE
            CALCULATETABLE(
                SUMMARIZE(
                    Products,
                    Products[Top Brands],  
                    Products[Business Name],   
                    "Value Share", IF(ISBLANK([Value Share]), 0, [Value Share]),
                    "WD", IF(ISBLANK([WD]), 0, [WD]),
                    "Net Sales", IF(ISBLANK([Net Sales]), 0, [Net Sales]),
                    "Trade Margin %", IF(ISBLANK([Trade Margin %]), 0, [Trade Margin %]),
                    "Business Name Sales Rate", IF(ISBLANK([Business Name Sales Rate]), 0,

In [ ]:
def execute_dax_query_with_retry(entity_name, area, hierby, entity_type, retries=3, delay=5):
    for attempt in range(retries):
        try:
            outputdic = {}
            dax_query = f"""
            EVALUATE
            CALCULATETABLE(
                SUMMARIZE(
                    Products,
                    Products[{prodORitem}],   
                    "Value Share", IF(ISBLANK([Value Share]), 0, [Value Share])
                ),
                Products[{hierby}] = "{entity_type}",
                TREATAS(
                    {p12m_dax},
                    Calendar[MonthYear]
                ),
                TREATAS(
                    {{"{entity_name}"}},
                    Market[{area}]
                ),
                FILTER(
                    'Scope', 
                    'Scope'[Scope] = "{hierby}"   
                )
            )   
            """
            with adodbapi.connect(conn_str) as conn:
                conn.timeout = 300  # Set connection timeout
                with conn.cursor() as cursor:
                    cursor.execute(dax_query)
                    columns = [desc[0] for desc in cursor.description]
                    data = cursor.fetchall()
                    
                    df = pd.DataFrame(data, columns=columns)
                    df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
                    df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

                    # Add Grand Total row
                    if not df.empty:
                        df = df.sort_values(by="Value Share", ascending=False).reset_index(drop=True)
                        df[f"Cumulative {prodORitem} Share"] = df["Value Share"].cumsum()
                    outputdic[f"{entity_type} | {entity_name}"] = df
                    print(f"Query executed successfully for {entity_type} {entity_name}.")
                    return outputdic
        except adodbapi.DatabaseError as db_error:
            print(f"Attempt {attempt + 1} failed for {entity_type} {entity_name}: {db_error}")
            if attempt < retries - 1:
                time.sleep(delay)  # Wait before retrying
            else:
                print(f"Max retries reached for {entity_type} {entity_name}.")
                raise
        except Exception as e:
            print(f"Unexpected error for {entity_type} {entity_name}: {e}")
            raise

# List of all entities to process
# Threaded execution
cumulative_share = {}
dfs_results = {}
ordered_keys = []
futures = []

with ThreadPoolExecutor(max_workers=8) as executor:
    future_to_key = {}

    for hierby, hier_values in hierarchy_levels:
        if isinstance(hier_values, list):
            for value in hier_values:
                for area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        key = f"{value} | {entity}"
                        ordered_keys.append(key)
                        future = executor.submit(execute_dax_query_with_retry, entity, area, hierby, value)
                        future_to_key[future] = key

    temp_results = {}
    for future in as_completed(future_to_key):
        key = future_to_key[future]
        try:
            result = future.result()
            temp_results.update(result)
        except Exception as e:
            print(f"Failed to retrieve result for {key}: {e}")

# Preserve original key order
for key in ordered_keys:
    if key in temp_results:
        dfs_results[key] = temp_results[key]

output_file = os.path.join(path, "cumulative_share.pkl")
with open(output_file, "wb") as f:
    pd.to_pickle(dfs_results, f)

print(f"All DataFrames saved to {output_file}.")


Query executed successfully for Manual Shave Men Walmart North.
Query executed successfully for Manual Shave Men Walmart East.
Query executed successfully for Manual Shave Men Walmart.
Query executed successfully for Manual Shave Men Walmart Southeast.
Query executed successfully for Manual Shave Men Walmart Southwest.
Query executed successfully for Manual Shave Men Sam's Corp.
Query executed successfully for Manual Shave Men Walmart West.
Query executed successfully for Manual Shave Men Bj's Corp.
Query executed successfully for Manual Shave Men Walmart Div1 Corp.
Query executed successfully for Manual Shave Men Walmart Nm Corp.
Query executed successfully for Manual Shave Men Walmart Sc Corp.
Query executed successfully for System Walmart East.
Query executed successfully for System Bj's Corp.
Query executed successfully for System Sam's Corp.
Query executed successfully for System Walmart.
Query executed successfully for System Walmart North.
Query executed successfully for System 

In [ ]:
def execute_dax_query(entity_name, area, hierby, brand=None, manuf=None):
    outputdic = {}
    manuf_filter = f'Products[{ManufOrTopC}]="{manuf}"' if manuf else ""
    brand_filter = f'Products[{BrandOrTopB}]="{brand}", Products[{ManufOrTopC}]="{client_manuf[0]}"' if brand else ""
    # Construct filter condition dynamically
    filters = ", ".join(filter(None, [manuf_filter, brand_filter, f'Products[Category] = "{categories[0]}"']))
    key = f"{brand} | {entity_name}" if brand else f"{manuf} | {entity_name}" if manuf else f"{entity_name}"
 
    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{hierby}],  
            "Av No SKUs", IF(ISBLANK([Av No SKUs]), 0, [Av No SKUs]),
            "SKU Share", IF(ISBLANK([SKU Share]), 0, [SKU Share]),
            "Value Share", IF(ISBLANK([Value Share]), 0, [Value Share])
        ),
        {filters},
        TREATAS(
            {p12m_dax},
            Calendar[MonthYear]
        ),
        TREATAS(
            {{"{entity_name}"}},
            Market[{area}]
        ),
        FILTER(
            'Scope',
            'Scope'[Scope] = "Category"  
        )
    )  
    """
 
    try:
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                columns = [desc[0] for desc in cursor.description]
                data = cursor.fetchall()
 
                df = pd.DataFrame(data, columns=columns)
                df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
                df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
 
                if not df.empty:
                    numeric_cols = df.select_dtypes(include='number').columns
                    total_values = {}
 
                    for col in numeric_cols:
                        if "price" in col.lower():  # Check if the column name contains "price" (case insensitive)
                            total_values[col] = df[col].mean()  # Take the average
                        else:
                            total_values[col] = df[col].sum()  # Take the sum
 
                    grand_total_row = pd.DataFrame([["Grand Total"] + [total_values.get(col, " ") for col in df.columns[1:]]], columns=df.columns)
                    df = pd.concat([df, grand_total_row], ignore_index=True)
                outputdic[key] = df
                print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name}: {e}")
   
    return outputdic
 
# Process data concurrently
def process_dax_queries(hierarchy_levels, entity_hierarchy, client_brands=None, client_manuf=None):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category":
                continue
           
            dfs_results = {}
            futures = []
           
            for area, entity_list in entity_hierarchy:
                for entity in entity_list:
                    if client_brands:
                        for brand in client_brands:
                            futures.append(executor.submit(execute_dax_query, entity, area, hierby, brand=brand))
                    elif client_manuf:
                        for manuf in client_manuf:
                            futures.append(executor.submit(execute_dax_query, entity, area, hierby, manuf=manuf))
                    else:
                        futures.append(executor.submit(execute_dax_query, entity, area, hierby))
 
            for future in as_completed(futures):
                result = future.result()
                dfs_results.update(result)
 
            # Determine output type
            output_type = "brands" if client_brands else ("manuf" if client_manuf else "total")
 
            # Save results
            output_path = rf"{path}\{hierby}_dfs_{output_type}.pkl"
            pd.to_pickle(dfs_results, output_path)
            print(f"All DataFrames saved to {output_path}.")
 
# Run the processing function for different levels
process_dax_queries(hierarchy_levels, entity_hierarchy)  # Total-level
process_dax_queries(hierarchy_levels, entity_hierarchy, client_brands=client_brands)  # Brand-level
process_dax_queries(hierarchy_levels, entity_hierarchy, client_manuf=client_manuf)  # Manufacturer-leve

Query executed successfully for Walmart East.
Query executed successfully for Walmart North.
Query executed successfully for Walmart Southeast.
Query executed successfully for Walmart Southwest.
Query executed successfully for Walmart West.
Query executed successfully for Sam's Corp.
Query executed successfully for Bj's Corp.
Query executed successfully for Walmart.
Query executed successfully for Walmart Div1 Corp.
Query executed successfully for Walmart Nm Corp.
Query executed successfully for Walmart Sc Corp.
All DataFrames saved to c:\Users\aleaa\Documents\Slide-Automate\Assortment Slide Duplicate\Assortment Datasets NewEX\Sector_dfs_total.pkl.
Query executed successfully for Walmart North.
Query executed successfully for Walmart East.
Query executed successfully for Walmart West.
Query executed successfully for Walmart Southeast.
Query executed successfully for Bj's Corp.
Query executed successfully for Walmart Southwest.
Query executed successfully for Sam's Corp.
Query executed 

In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Script started at: Wed Jul  9 17:07:04 2025
Script ended at: Wed Jul  9 17:10:36 2025
Elapsed time: 212.45 seconds


In [ ]:
loaded_data = {}
datasets_path =os.getcwd()+"\\Assortment Datasets NewEX\\"
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)